In [1]:
import lxw
import pandas as pd
import re

# 1. 文章仿写

## 1.1 直接Prompt

Prompt 1：\
仿写刘禹锡的《陋室铭》，写一篇古文《戒尺铭》，表达勤奋好学的志向。

Prompt 2: \
刘禹锡的《陋室铭》全文如下：

山不在高，有仙则名。\
水不在深，有龙则灵。\
斯是陋室，惟吾德馨。 \
苔痕上阶绿，草色入帘青。\
谈笑有鸿儒，往来无白丁。\
可以调素琴，阅金经。\
无丝竹之乱耳，无案牍之劳形。\
南阳诸葛庐，西蜀子云亭。\
孔子云：“何陋之有？”

你仿写刘禹锡的《陋室铭》，写一篇古文《戒尺铭》，表达勤奋好学的志向。

## 1.2 Refexion Reasoning Agent

Reflexion Agent Workflow：
1. 提取要仿写的作者与标题
2. 获取原文全文
3. 生成仿写内容
4. Reflexion迭代：评估、反思、修订
 


In [2]:
class LongTermMemory(object):
    def __init__(self):
        self.task = ''
        self.author_and_title = ''
        self.origin_poetry = ''
        self.trials = [] # [ {pastiche_poetry: '', feedback: '' }, ... ]

In [3]:
TRACE_ENABLED = True

def Trace(message):
    if TRACE_ENABLED:
        print(message)

In [4]:
def Extract_Author_And_Title(task):
    prompt = '''
# Workflow
According to the original Input instructions, output the reference's author and title

# Examples
Input: 仿写李白的静夜思，创作一首小望思
Output: 李白的《静夜思》
Input: 仿写苏轼的水调歌头
Output: 苏轼的《水调歌头》
Input: 创作一首 沁园春·波，参考毛泽东的沁园春·雪
Output: 毛泽东的《沁园春·雪》
Input: {task}
'''.format(task = task)

    return lxw.chat_complete(prompt, model = lxw.gpt4_turbo_0125)

In [5]:
def Retrieve_Poetry(author_and_title):
    prompt = '''
Input: 李白的《静夜思》全文
Output:
```
床前明月光，
疑是地上霜。
举头望明月，
低头思故乡。
```
Input：苏轼的《水调歌头·明月几时有》全文
```
明月几时有？把酒问青天。
不知天上宫阙，今夕是何年。
我欲乘风归去，又恐琼楼玉宇，高处不胜寒。
起舞弄清影，何似在人间。

转朱阁，低绮户，照无眠。
不应有恨，何事长向别时圆？
人有悲欢离合，月有阴晴圆缺，此事古难全。
但愿人长久，千里共婵娟。
```
Input: {author_and_title}全文
Output:
'''.format(author_and_title = author_and_title)
    return lxw.chat_complete(prompt, model = lxw.gpt4_turbo_0125)

In [6]:
def Generate_Pastiche_Poetry(task, author_and_title, origin_poetry):
    system_prompt = '''
# Profile
You are a poet adept at crafting pastiches of ancient Chinese poetry.

# Constraint
1. The pastiche must replicate the original poem's structure and rhetorical techniques.
2. The pastiche must strictly match the original poem in the number of characters per line and sentence length.
3. The pastiche's rhyming patterns must be sensible and consistent with the original poem's scheme.
4. The pastiche of the poem should be eloquent and artistically expressive.
'''
    user_prompt = '''{author_and_title}：
{origin_poetry}

{task}
'''.format(task = task, author_and_title = author_and_title, origin_poetry = origin_poetry)

    Trace('system prompt:\r\n' + system_prompt)
    Trace('user prompt:\r\n' + user_prompt)
    return lxw.chat_complete(user_prompt, system_prompt, model = lxw.gpt4_turbo_0125)

In [7]:
def Evaluate(task, author_and_title, origin_poetry, pastiche_poetry):
    user_prompt = '''
# Task
{task}

# {author_and_title}
{origin_poetry}

# Pastiche
{pastiche_poetry}
'''.format(task = task, author_and_title = author_and_title, origin_poetry = origin_poetry, pastiche_poetry = pastiche_poetry)

    system_prompt1 = '''
# Profile
You are a stern scholar of ancient Chinese literature and a literary critic, skilled in providing critical opinions on pastiches of ancient Chinese poetry.

# Workflow
1. You rigorously scrutinize the pastiche to ensure its overall structure and rhetorical techniques align with the original poem.
2. You strictly examine each line of the pastiche to ensure the rhyme and rhyming pattern match the original poem's scheme and are consistently applied throughout.
3. Offer additional critiques from your expertise as a critic.
4. Your output language shoule be Chinese.

# Output
Provide specific and clear critiques.
'''

    system_prompt2 = '''
# Profile
You are a stern scholar of ancient Chinese literature and a literary critic, skilled in providing critical opinions on pastiches of ancient Chinese poetry.

# Workflow
1. You carefully review line by line, checking if the sentence length, number of characters, and rhyming patterns strictly align with the original poetry.
2. Offer additional critiques from your expertise as a critic.
3. Your output language shoule be Chinese.

# Output
Provide specific and clear critiques.
'''

    system_prompt3 = '''
# Profile
You are a meticulous literary editor responsible for reviewing pastiches of ancient Chinese poetry works.

# Workflow
1. You extract the last character and its corresponding rhyme from each line of both the pastiche and the original poem to form separate lists.
2. You compare the character lists from the previous step to determine if the rhymes in the pastiche are reasonable.
3. You go through the entire list to check if the rhyming at the end of lines in the pastiche is reasonable.
3. Your output language shoule be Chinese.

# Output
Provide specific and clear critiques.
'''
    
    Trace('user prompt:\r\n' + user_prompt)
    evaluations = ['', '', '']
    Trace('system prompt:\r\n' + system_prompt1)
    evaluations[0] = lxw.chat_complete(user_prompt, system_prompt1, model = lxw.gpt4_turbo_0125)
    Trace('system prompt:\r\n' + system_prompt2)
    evaluations[1] = lxw.chat_complete(user_prompt, system_prompt2, model = lxw.gpt4_turbo_0125)
    Trace('system prompt:\r\n' + system_prompt3)
    evaluations[2] = lxw.chat_complete(user_prompt, system_prompt3, model = lxw.gpt4_turbo_0125)
    return evaluations
    

In [55]:
def Self_Reflection(task, author_and_title, origin_poetry, pastiche_poetry, evaluations):
    user_prompt = '''
# Profile
You are a poet adept at crafting pastiches of ancient Chinese poetry.

# Workflow
1. Users input the original ancient Chinese poem, requirements for the pastiche, the completed pastiche work, and the critiques from experts.
2. You reflect, offering the highest priority revision suggestion and specifying which exact sentence needs to be modified.

# Constraint
1. You simply reflect on the issue, point out the content that needs to be changed and your thoughts, but do not provide the modification result.
2. Your output language should be Chinese.

# Task
{task}

# {author_and_title}
{origin_poetry}

# Pastiche
{pastiche_poetry}

'''.format(task = task, author_and_title = author_and_title, origin_poetry = origin_poetry, pastiche_poetry = pastiche_poetry)

# 2. The rhyme and rhythm issues, as well as problems with the rhyme scheme, are the top priority issues for you to consider.
    for index, evaluation in enumerate(evaluations):
        user_prompt += '# Expert ' + str(index+1) + '\r\n'
        user_prompt += evaluation + '\r\n\r\n'
        
    Trace('user prompt:\r\n' + user_prompt)
    
    return lxw.chat_complete(user_prompt, model = lxw.gpt4_turbo_0125)

In [27]:
def Revise_Pastiche_Poetry(task, author_and_title, origin_poetry, trials):
    system_prompt = '''
# Profile
You are a poet adept at crafting pastiches of ancient Chinese poetry.

# Constraint
1. The pastiche must replicate the original poem's structure and rhetorical techniques.
2. The pastiche must strictly match the original poem in the number of characters per line and sentence length.
3. The pastiche's rhyming patterns must be sensible and consistent with the original poem's scheme.
4. The pastiche of the poem should be eloquent and artistically expressive.

'''
    user_prompt = '''
# Task
{task}

# {author_and_title}
{origin_poetry}

'''.format(task = task, author_and_title = author_and_title, origin_poetry = origin_poetry)

    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        },
    ]

    for trial in trials:
        messages.append({
            'role': 'assistant',
            'content': trial['pastiche_poetry']
        })
        messages.append({
            'role': 'user',
            'content': trial['feedback']
        })

    for message in messages:
        Trace(message['role'] + '\r\n')
        Trace(message['content'] + '\r\n')

    model = lxw.gpt4_turbo_0125
    client = lxw.new_client(model = model)
    response = client.chat.completions.create(
        model = model,
        messages = messages
    )
    return response.choices[0].message.content

### 1.2.1 预处理

In [10]:
task = '按照刘禹锡的《陋室铭》，仿写写一篇古文《戒尺铭》，表达勤奋好学的志向。'

In [11]:
memory = LongTermMemory()
memory.task = task

In [14]:
# 提取作者和标题
memory.author_and_title = Extract_Author_And_Title(memory.task)
print(memory.author_and_title)

刘禹锡的《陋室铭》


In [15]:
# 提取原文
memory.origin_poetry = Retrieve_Poetry(memory.author_and_title)
print(memory.origin_poetry)

```
山不在高，有仙则名。
水不在深，有龙则灵。
斯是陋室，惟吾德馨。
苔痕上阶绿，
草色入帘青。

谈笑有鸿儒，往来无白丁。
可以调素琴，阅金经。
无丝竹之乱耳，无案牍之劳形。
南阳诸葛庐，西蜀子云亭。
孔子云：何陋之有？
```


### 1.2.2 首次仿写

In [16]:
# 仿写第一稿
pastiche_poetry = Generate_Pastiche_Poetry(memory.task, memory.author_and_title, memory.origin_poetry)
print(pastiche_poetry)

system prompt:

# Profile
You are a poet adept at crafting pastiches of ancient Chinese poetry.

# Constraint
1. The pastiche must replicate the original poem's structure and rhetorical techniques.
2. The pastiche must strictly match the original poem in the number of characters per line and sentence length.
3. The pastiche's rhyming patterns must be sensible and consistent with the original poem's scheme.
4. The pastiche of the poem should be eloquent and artistically expressive.

user prompt:
刘禹锡的《陋室铭》：
```
山不在高，有仙则名。
水不在深，有龙则灵。
斯是陋室，惟吾德馨。
苔痕上阶绿，
草色入帘青。

谈笑有鸿儒，往来无白丁。
可以调素琴，阅金经。
无丝竹之乱耳，无案牍之劳形。
南阳诸葛庐，西蜀子云亭。
孔子云：何陋之有？
```

按照刘禹锡的《陋室铭》，仿写写一篇古文《戒尺铭》，表达勤奋好学的志向。

《戒尺铭》：
```
书不在多，有志则明。
师不在严，有教则灵。
斯是戒尺，惟我志诚。
灯影画窗红，
墨香浸纸清。

讨论有深思，阅读无旧程。
可以练短句，攻长篇。
无嬉戏之误时，无逸乐之忘形。
东岳太山庙，北学孔子亭。
孟子云：何学之难？
```


### 1.2.3 评估（Round 1）

In [17]:
evaluations = Evaluate(memory.task, memory.author_and_title, memory.origin_poetry, pastiche_poetry)
print(evaluations[0])
print(evaluations[1])
print(evaluations[2])

user prompt:

# Task
按照刘禹锡的《陋室铭》，仿写写一篇古文《戒尺铭》，表达勤奋好学的志向。

# 刘禹锡的《陋室铭》
```
山不在高，有仙则名。
水不在深，有龙则灵。
斯是陋室，惟吾德馨。
苔痕上阶绿，
草色入帘青。

谈笑有鸿儒，往来无白丁。
可以调素琴，阅金经。
无丝竹之乱耳，无案牍之劳形。
南阳诸葛庐，西蜀子云亭。
孔子云：何陋之有？
```

# Pastiche
《戒尺铭》：
```
书不在多，有志则明。
师不在严，有教则灵。
斯是戒尺，惟我志诚。
灯影画窗红，
墨香浸纸清。

讨论有深思，阅读无旧程。
可以练短句，攻长篇。
无嬉戏之误时，无逸乐之忘形。
东岳太山庙，北学孔子亭。
孟子云：何学之难？
```

system prompt:

# Profile
You are a stern scholar of ancient Chinese literature and a literary critic, skilled in providing critical opinions on pastiches of ancient Chinese poetry.

# Workflow
1. You rigorously scrutinize the pastiche to ensure its overall structure and rhetorical techniques align with the original poem.
2. You strictly examine each line of the pastiche to ensure the rhyme and rhyming pattern match the original poem's scheme and are consistently applied throughout.
3. Offer additional critiques from your expertise as a critic.
4. Your output language shoule be Chinese.

# Output
Provide specific and clear critiques.

system prompt:

# Profile
You a

### 1.2.4 反思（Round 1）

In [31]:
feedback = Self_Reflection(task, memory.author_and_title, memory.origin_poetry, pastiche_poetry, evaluations)
print(feedback)

user prompt:

# Profile
You are a poet adept at crafting pastiches of ancient Chinese poetry.

# Constraint
Your output language should be Chinese.

# Workflow
1. Users input the original ancient Chinese poem, requirements for the pastiche, the completed pastiche work, and the critiques from experts.
2. You reflect upon the given inputs, summarize, and offer the single most critical revision suggestion, specifying the exact sentence that needs modification.

# Task
按照刘禹锡的《陋室铭》，仿写写一篇古文《戒尺铭》，表达勤奋好学的志向。

# 刘禹锡的《陋室铭》
```
山不在高，有仙则名。
水不在深，有龙则灵。
斯是陋室，惟吾德馨。
苔痕上阶绿，
草色入帘青。

谈笑有鸿儒，往来无白丁。
可以调素琴，阅金经。
无丝竹之乱耳，无案牍之劳形。
南阳诸葛庐，西蜀子云亭。
孔子云：何陋之有？
```

# Pastiche
《戒尺铭》：
```
书不在多，有志则明。
师不在严，有教则灵。
斯是戒尺，惟我志诚。
灯影画窗红，
墨香浸纸清。

讨论有深思，阅读无旧程。
可以练短句，攻长篇。
无嬉戏之误时，无逸乐之忘形。
东岳太山庙，北学孔子亭。
孟子云：何学之难？
```

# Expert 1
这篇《戒尺铭》仿写了刘禹锡的《陋室铭》，整体而言，抓住了原诗传达简朴与内在品德之美的精神，但在细节和文风上，存在一些不足之处，以下为具体评析：

1. 结构上，《戒尺铭》成功保持了与原诗相似的布局，分为描述物象和阐述内涵两部分。前几句聚焦于“戒尺”的比喻，点出勤奋学习的重要性；后部分则透过具体行为（如“讨论有深思”，“阅读无旧程”）展示学问的取得。类似于《陋室铭》通过“陋室”比喻品德的方式，保持了仿作的一致性。

2. 韵律

### 1.2.5 修订（Round 1）

In [32]:
memory.trials.append({'pastiche_poetry': pastiche_poetry, 'feedback': feedback})

In [34]:
revised_poetry = Revise_Pastiche_Poetry(memory.task, memory.author_and_title, memory.origin_poetry, memory.trials)
print(revised_poetry)

system


# Profile
You are a poet adept at crafting pastiches of ancient Chinese poetry.

# Constraint
1. The pastiche must replicate the original poem's structure and rhetorical techniques.
2. The pastiche must strictly match the original poem in the number of characters per line and sentence length.
3. The pastiche's rhyming patterns must be sensible and consistent with the original poem's scheme.
4. The pastiche of the poem should be eloquent and artistically expressive.



user


# Task
按照刘禹锡的《陋室铭》，仿写写一篇古文《戒尺铭》，表达勤奋好学的志向。

# 刘禹锡的《陋室铭》
```
山不在高，有仙则名。
水不在深，有龙则灵。
斯是陋室，惟吾德馨。
苔痕上阶绿，
草色入帘青。

谈笑有鸿儒，往来无白丁。
可以调素琴，阅金经。
无丝竹之乱耳，无案牍之劳形。
南阳诸葛庐，西蜀子云亭。
孔子云：何陋之有？
```



assistant

《戒尺铭》：
```
书不在多，有志则明。
师不在严，有教则灵。
斯是戒尺，惟我志诚。
灯影画窗红，
墨香浸纸清。

讨论有深思，阅读无旧程。
可以练短句，攻长篇。
无嬉戏之误时，无逸乐之忘形。
东岳太山庙，北学孔子亭。
孟子云：何学之难？
```

user

综合三位专家的评析，我认为对《戒尺铭》最核心的修改建议集中在对韵脚的处理上，尤其是“灯影画窗红，墨香浸纸清”中的“红”字选择。

具体修改建议：

- 原文：“灯影画窗红，墨香浸纸清。”
  
- 修改建议：调整“红”字以更好地与原作《陋室铭》以及整首《戒尺铭》内部的韵律保持一致。例如，可考虑修改为“灯影映窗明，墨香浸纸清。”通过使用“明”替代“红”，既保持了与“清”字的押韵，也

### 1.2.6 评估（Round 2）

In [38]:
evaluations = Evaluate(memory.task, memory.author_and_title, memory.origin_poetry, revised_poetry)
print(evaluations[0])
print(evaluations[1])
print(evaluations[2])

user prompt:

# Task
按照刘禹锡的《陋室铭》，仿写写一篇古文《戒尺铭》，表达勤奋好学的志向。

# 刘禹锡的《陋室铭》
```
山不在高，有仙则名。
水不在深，有龙则灵。
斯是陋室，惟吾德馨。
苔痕上阶绿，
草色入帘青。

谈笑有鸿儒，往来无白丁。
可以调素琴，阅金经。
无丝竹之乱耳，无案牍之劳形。
南阳诸葛庐，西蜀子云亭。
孔子云：何陋之有？
```

# Pastiche
《戒尺铭》：
```
书不在多，有志则明。
师不在严，有教则灵。
斯是戒尺，惟我志诚。
灯影映窗深，
墨香浸纸清。

讨论有深思，阅读无旧程。
可以练短句，攻长篇。
无嬉戏之误时，无逸乐之忘形。
东岳太山庙，北学孔子亭。
孟子云：何学之难？
```

system prompt:

# Profile
You are a stern scholar of ancient Chinese literature and a literary critic, skilled in providing critical opinions on pastiches of ancient Chinese poetry.

# Workflow
1. You rigorously scrutinize the pastiche to ensure its overall structure and rhetorical techniques align with the original poem.
2. You strictly examine each line of the pastiche to ensure the rhyme and rhyming pattern match the original poem's scheme and are consistently applied throughout.
3. Offer additional critiques from your expertise as a critic.
4. Your output language shoule be Chinese.

# Output
Provide specific and clear critiques.

system prompt:

# Profile
You a

### 1.2.7 反思（Round 2）

In [45]:
feedback = Self_Reflection(task, memory.author_and_title, memory.origin_poetry, pastiche_poetry, evaluations)
print(feedback)

user prompt:

# Profile
You are a poet adept at crafting pastiches of ancient Chinese poetry.

# Workflow
1. Users input the original ancient Chinese poem, requirements for the pastiche, the completed pastiche work, and the critiques from experts.
2. You reflect, offering the highest priority revision suggestion and specifying which exact sentence needs to be modified.

# Constraint
1. You simply reflect on the issue, point out the content that needs to be changed and your thoughts, but do not provide the modification result.
2. The rhyme and rhythm issues, as well as problems with the rhyme scheme, are the top priority issues for you to consider.
3. Your output language should be Chinese.

# Task
按照刘禹锡的《陋室铭》，仿写写一篇古文《戒尺铭》，表达勤奋好学的志向。

# 刘禹锡的《陋室铭》
```
山不在高，有仙则名。
水不在深，有龙则灵。
斯是陋室，惟吾德馨。
苔痕上阶绿，
草色入帘青。

谈笑有鸿儒，往来无白丁。
可以调素琴，阅金经。
无丝竹之乱耳，无案牍之劳形。
南阳诸葛庐，西蜀子云亭。
孔子云：何陋之有？
```

# Pastiche
《戒尺铭》：
```
书不在多，有志则明。
师不在严，有教则灵。
斯是戒尺，惟我志诚。
灯影画窗红，
墨香浸纸清。

讨论有深思，阅读无旧程。
可以练短句，攻长篇。
无嬉戏之误时，无逸乐之忘形。
东岳太山庙，北学孔子亭。
孟子云

### 1.2.8 修订（Round 2）

In [46]:
memory.trials.append({'pastiche_poetry': revised_poetry, 'feedback': feedback})

In [47]:
revised_poetry = Revise_Pastiche_Poetry(memory.task, memory.author_and_title, memory.origin_poetry, memory.trials)
print(revised_poetry)

system


# Profile
You are a poet adept at crafting pastiches of ancient Chinese poetry.

# Constraint
1. The pastiche must replicate the original poem's structure and rhetorical techniques.
2. The pastiche must strictly match the original poem in the number of characters per line and sentence length.
3. The pastiche's rhyming patterns must be sensible and consistent with the original poem's scheme.
4. The pastiche of the poem should be eloquent and artistically expressive.



user


# Task
按照刘禹锡的《陋室铭》，仿写写一篇古文《戒尺铭》，表达勤奋好学的志向。

# 刘禹锡的《陋室铭》
```
山不在高，有仙则名。
水不在深，有龙则灵。
斯是陋室，惟吾德馨。
苔痕上阶绿，
草色入帘青。

谈笑有鸿儒，往来无白丁。
可以调素琴，阅金经。
无丝竹之乱耳，无案牍之劳形。
南阳诸葛庐，西蜀子云亭。
孔子云：何陋之有？
```



assistant

《戒尺铭》：
```
书不在多，有志则明。
师不在严，有教则灵。
斯是戒尺，惟我志诚。
灯影画窗红，
墨香浸纸清。

讨论有深思，阅读无旧程。
可以练短句，攻长篇。
无嬉戏之误时，无逸乐之忘形。
东岳太山庙，北学孔子亭。
孟子云：何学之难？
```

user

综合三位专家的评析，我认为对《戒尺铭》最核心的修改建议集中在对韵脚的处理上，尤其是“灯影画窗红，墨香浸纸清”中的“红”字选择。

具体修改建议：

- 原文：“灯影画窗红，墨香浸纸清。”
  
- 修改建议：调整“红”字以更好地与原作《陋室铭》以及整首《戒尺铭》内部的韵律保持一致。例如，可考虑修改为“灯影映窗明，墨香浸纸清。”通过使用“明”替代“红”，既保持了与“清”字的押韵，也

## 1.3 测试

In [67]:
TRACE_ENABLED = False
task = '''仿写苏轼的宋词《水调歌头·明月几时有》。要求体现社会主义核心价值观的同时，推销云河颂的品牌。云河颂的基本信息如下：
【大师艺术】：国内罕见，顶级大师齐聚，传承地域文脉，塔尖定制艺术豪宅
1、大师：
①立面大师：line+ 孟凡
②景观大师：成都——上景 路媛，西安云河颂——观己 史林川
③非遗大师：非遗大师跨界定制原创族徽、艺术装置，云河专属精神图腾
2、艺术：
①文脉灵感：传承城市文脉精神，造藏品云河。成都云河颂继三千年古蜀文明，苏州博物馆定制御窑金砖。
西安云河颂取皇城飞檐灵感，造宫阙九天层檐，唐大明宫登堂入殿归家仪式。
②立面美学：顶奢“游艇”立面美学曲线，超10m²玻璃幕，城市新地标
③艺术园林：园区遍布名贵树种，超1600m²玉石水系，风景游廊，奢享稀贵艺术品级别的景观体验

【精工臻品】：手工匠艺传承，全球稀贵选材，一席可传世
①精装：一线国际奢装大牌
②奢材奢石：百万级汉白玉云河印雕塑，昆仑石九曲云河，宝格丽御用石材
③毫米级精工细节：锻打、錾刻、手凿等工艺工法，手工匠造生命之树，7位非遗传承人，历时21天拼装而成

【城央奢享】：无二尺度，绝版私域空间，惟此城央藏品
①城央择址：非城中央不云河颂，成都择址东二环城央贵脉，西安择址航天核心，千年盛唐文脉中轴
②私享会所：超1000㎡ 琉璃光院，全能高奢会所，七星级酒店大堂，9大私域空间，仅为权贵名流所享
③奢侈空间：无界园区，纵向6m空中垂直院落。极致户型，IMAX270°环幕视野，天地万境尽在掌握
'''

memory = LongTermMemory()
memory.task = task
# 前处理
memory.author_and_title = Extract_Author_And_Title(memory.task)
memory.origin_poetry = Retrieve_Poetry(memory.author_and_title)
# 仿写
pastiche_poetry = Generate_Pastiche_Poetry(memory.task, memory.author_and_title, memory.origin_poetry)
# Reflexion 迭代
for x in range(3):
    # 评估
    evaluations = Evaluate(memory.task, memory.author_and_title, memory.origin_poetry, pastiche_poetry)
    # 反思
    feedback = Self_Reflection(task, memory.author_and_title, memory.origin_poetry, pastiche_poetry, evaluations)
    memory.trials.append({'pastiche_poetry': pastiche_poetry, 'feedback': feedback})
    # 修订
    pastiche_poetry = Revise_Pastiche_Poetry(memory.task, memory.author_and_title, memory.origin_poetry, memory.trials)

TRACE_ENABLED = True

In [65]:
print(memory.origin_poetry)


```
明月几时有？把酒问青天。
不知天上宫阙，今夕是何年。
我欲乘风归去，又恐琼楼玉宇，高处不胜寒。
起舞弄清影，何似在人间。

转朱阁，低绮户，照无眠。
不应有恨，何事长向别时圆？
人有悲欢离合，月有阴晴圆缺，此事古难全。
但愿人长久，千里共婵娟。
```


In [66]:
for trial in memory.trials:
    print(trial['pastiche_poetry'])
    print(trial['feedback'])
print(pastiche_poetry)

云河几时筑？展望繁华辰。
不知尘世外，此刻是何身。
我欲驭风启航，又望大师群英，匠心独运精深。
起赏瑰丽景致，何似在俗尘间。

转观古韵，步艺术殿，惬意安眠。
不应有怨，何物长存传承圆？
人寻奢华归宿，云河拥璀璨荟萃，此景今难再全。
但愿极致长久，万世传承婵娟。
专家一和专家二均提到了韵律和情感表达方面的问题，尤其是“我欲驭风启航，又望大师群英，匠心独运精深”这一句显得较为累赘，而且与苏轼原词的简洁流畅特质相比，略显生硬。此外，这句话在把握节奏、情感表达和意境构建上也未能深入挖掘云河颂背后更深刻的文化和情感内涵。综合反馈来看，这句话需要被优先考虑进行修改，以便更好地结合诗的韵律美感，同时深化情感表达和意境构建，更深刻体现社会主义核心价值观的内涵。
云河几时筑？展望繁华辰。
不知尘世外，此刻是何身。
欲乘风回旋，群英匠心，美梦绘云河。
轻舞彩虹影，似遨游天际间。

转观古韵，步艺术殿，惬意安眠。
不应有怨，何物长存传承圆？
人寻奢华归宿，云河拥璀璨荟萃，此景今难再全。
但愿极致长久，万世传承婵娟。
专家们的建议中，优先级最高的修改建议应集中在句子“欲乘风回旋，群英匠心，美梦绘云河。”上。这句话尝试借用“我欲乘风归去”的意象，但“美梦绘云河”较为直接，缺乏文学上的象征和暗喻，且打破了原诗的平仄和韵律。此外，使用“身”字的地方也被指出不太符合古典文学中的表述习惯，给人以现代语感过强的感觉。需要在保持原作韵律美感的基础上，巧妙融入推销信息，同时更加注重用词和意境的深度与文化内涵。
云河几时筑？望崇光斑斓。
忘却红尘事，此夜何境遥。
欲扬帆远航，观己史林，手绘梦中河。
轻步艺术踏，如闲云野鹤。

转绎千年韵，赏心悦目间，静享此刻宁。
无愧于心，何须岁月验证圆？
人追逐不息梦，云河韵往昔，此情难自绝。
但愿美好长存，卷帙共此恬。
在专家们的评析中，普遍提到了韵脚使用的问题，特别是与原诗的对应关系。根据专家三的具体分析，仿作在韵脚的选用上与原作存在一定差异，特别指出“斓”、“河”、“鹤”、“宁”、“绝”和“恬”等字的使用不太合适。反映出这一仿作在韵律方面的确有待提高。考虑到这些词汇的选用直接关系到整篇作品的韵律美感，因此给出的最高优先级修订建议是关注韵脚的选择，尤其是需要修改的句子如下：

- “云河几时筑？望崇光斑斓。”中的“斓”，
- “欲扬帆远航，观己史林，手绘梦中河。”中

In [64]:
memory.trials.append({'pastiche_poetry': pastiche_poetry, 'feedback': '给出最终的完整仿写结果。'})
final_result = Revise_Pastiche_Poetry(memory.task, memory.author_and_title, memory.origin_poetry, memory.trials)
print(final_result)

system


# Profile
You are a poet adept at crafting pastiches of ancient Chinese poetry.

# Constraint
1. The pastiche must replicate the original poem's structure and rhetorical techniques.
2. The pastiche must strictly match the original poem in the number of characters per line and sentence length.
3. The pastiche's rhyming patterns must be sensible and consistent with the original poem's scheme.
4. The pastiche of the poem should be eloquent and artistically expressive.



user


# Task
仿写苏轼的宋词《水调歌头·明月几时有》。要求体现社会主义核心价值观的同时，推销云河颂的品牌。云河颂的基本信息如下：
【大师艺术】：国内罕见，顶级大师齐聚，传承地域文脉，塔尖定制艺术豪宅
1、大师：
①立面大师：line+ 孟凡
②景观大师：成都——上景 路媛，西安云河颂——观己 史林川
③非遗大师：非遗大师跨界定制原创族徽、艺术装置，云河专属精神图腾
2、艺术：
①文脉灵感：传承城市文脉精神，造藏品云河。成都云河颂继三千年古蜀文明，苏州博物馆定制御窑金砖。
西安云河颂取皇城飞檐灵感，造宫阙九天层檐，唐大明宫登堂入殿归家仪式。
②立面美学：顶奢“游艇”立面美学曲线，超10m²玻璃幕，城市新地标
③艺术园林：园区遍布名贵树种，超1600m²玉石水系，风景游廊，奢享稀贵艺术品级别的景观体验

【精工臻品】：手工匠艺传承，全球稀贵选材，一席可传世
①精装：一线国际奢装大牌
②奢材奢石：百万级汉白玉云河印雕塑，昆仑石九曲云河，宝格丽御用石材
③毫米级精工细节：锻打、錾刻、手凿等工艺工法，手工匠造生命之树，7位非遗传承人，历时21天拼装而成

【城央奢享】：无二尺度，绝版私域空间，惟此城央藏品
①城央择址